# Passive microrheology Analysis
This program analyzes the trajectories of a trapped particle inside an optical tweezer to obtain the rheological moduli G' and G'' of the surrounding medium. The calculation is done following the works of Tassieri et al. see the references below: 
* Tassieri, M. Microrheology with Optical Tweezers: Principles and Applications.  (CRC Press, 2016).
* Preece, D. et al. Optical tweezers: wideband microrheology. Journal of optics 13, 044022 (2011).
* Tassieri, M., Evans, R., Warren, R. L., Bailey, N. J. & Cooper, J. M. Microrheology with optical tweezers: data analysis. New Journal of Physics 14, 115032 (2012).
* Tassieri, M. et al. Measuring storage and loss moduli using optical tweezers: Broadband microrheology. Physical Review E 81, 026308 (2010).

This program is published as a part of associated code for the manuscript "*Programmable Viscoelasticity in Protein-RNA Condensates with Disordered Sticker-Spacer Polypeptides*" by Alshareedah and coworkers. 

Program author: Ibraheem Alshareedah. 
Last updated: Oct 6th 2021

### Importing necessary libraries and defining the functions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lumicks import pylake
import math
from multipletau import autocorrelate
import array
import lmfit
import scipy as sp
import numpy as np
from scipy.interpolate import interp1d
from obspy.signal.detrend import spline
from os import listdir
from os.path import isfile, join


%matplotlib inline





def ia_autocorr(xpos,ypos,bins,time_interval):
    
    n=np.arange(0,bins)
    k=0
    lagtimes=np.zeros(len(n))
    for i in n:
        lagtimes[k]=math.ceil(1.45**i)
        k=k+1
    k=0
    Ax=np.zeros(len(n))
    Ay=np.zeros(len(n))
    variancey = np.var(ypos,dtype=np.float64)
    variancex = np.var(xpos,dtype=np.float64)
    for dt in lagtimes:
        dt=int(dt)
        Ax[k] = np.mean(xpos[1+dt:len(xpos)-1]*xpos[1:len(xpos)-1-dt])/variancex
        Ay[k] = np.mean(ypos[1+dt:len(ypos)-1]*ypos[1:len(ypos)-1-dt])/variancey


        k=k+1
    return lagtimes*time_interval, Ax, Ay

def compute_autocorrelation(xpos,ypos,dt):
    xpos=xpos-np.mean(xpos)
    ypos=ypos-np.mean(ypos)
    corr_datax = autocorrelate(xpos*1e-9, m=16,deltat=dt, normalize=False)
    corr_datay = autocorrelate(ypos*1e-9, m=16,deltat=dt, normalize=False)
    var1x = sum(xpos*xpos*1e-18)
    var1y = sum(ypos*ypos*1e-18)
    tau=corr_datax[:,0]
    Ax=corr_datax[:,1]/var1x
    Ay = corr_datay[:,1]/var1y    
    return tau, Ax,Ay


def get_stiffness(t,x,y,T):
    variancey = np.var(y,dtype=np.float64)
    variancex = np.var(x,dtype=np.float64)
    k = 1.38064852e-23
    Temp = T+273.15 # 293.15 #20 C 
    kappax = 1e18*k*Temp/variancex 
    kappay = 1e18*k*Temp/variancey 

    return kappax,kappay


def PMR(taunew,Atimenew,g0,gdotinf):

        #calculate -Afreq*omega**2
    
    import math

    hpow=math.log10(1/taunew[0] )
    lpow = math.log10(1/taunew[len(taunew)-1])

    omegas=np.logspace(lpow,hpow,300)
    Afft = np.zeros(300,dtype=complex)
    Afreq = np.zeros(300,dtype=complex)

 
    #Atimenew=[1:len(Atimenew)-1]
    #taunew=[1:len(taunew)-1]
    dA=np.diff(Atimenew)
    dt=np.diff(taunew)
   
    gdot=dA/dt


    k=0
    for omega in omegas:
        Afft[k]=1j*omega*g0 +(1-np.exp(-1j*omega*taunew[0]))*(Atimenew[0]-g0)/taunew[0] + gdotinf*np.exp(-1j*omega*taunew[len(taunew)-1])

        dexp = np.diff(np.exp(-1j*omega*taunew))
        Afft[k]=Afft[k]+sum(-dexp*dA/dt)
        Afreq[k]=-Afft[k]/(omega)**2
        k=k+1
     

                                                                                                            
    return omegas,Afreq



from pylab import *

def myplot(xlabel,ylabel):
    
    rc('axes', linewidth=1.5)
    fontsize = 14

    ax=gca()
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        #tick.label1.set_fontweight('bold')
        tick.linewidth=1.5
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        #tick.label1.set_fontweight('bold')
    plt.xlabel(xlabel,fontsize=18)
    plt.ylabel(ylabel,fontsize=18)
    plt.legend(frameon=False)
    grid()

# Opening the data file
Here, the code is written to handle .h5 files obtained from Lumicks bluelake software that operates the C-trap correlative optical tweezer-confocal microscopy system. the code can be edited to open other files. The needed data is the X-coordinates, Y-coordinates, and time arrays. 

In [ ]:
# List the h5 files in the directory


files = listdir()
scans = []
fraps=[]
for file in files:
    if '.h5' in file  :
        scans.append(file)
index = np.arange(len(scans))
insca = zip(list(index),scans)
list(insca)

In [ ]:
# Pick a file to analyze
index_to_analyze =1
fname=scans[index_to_analyze]
fname=fname[0:len(fname)-3]
file1 = pylake.File(fname+'.h5')
#file2 = pylake.File("data/20180611-161216 Kymograph 6.h5")
#print(file1)

beadx = file1['Bead position']['Bead 1 X']
beadxx = beadx.data
t = beadx.timestamps
t= (t-t[0])/1e9
beady = file1['Bead position']['Bead 1 Y']
beadyy = beady.data
#plt.plot(beadxx,beadyy)
xpos=beadxx-beadxx[0]
ypos=beadyy-beadyy[0]

In [ ]:
#correct for drifiting
spline(xpos, order=2, dspline=1000000, plot=True)  
spline(ypos, order=2, dspline=1000000, plot=True) 

# Calculating the Autocorrelation functions in x and y
We use multipletau.autocorrelate and a user-defined ia_correlate to compute two versions of the autocorrelation function based on the time-lag spacing. multipletau.autocorrelate() uses a 2^n binning of lagtimes while ia_autocorrelate uses a 1.5^n binning of lagtimes. The latter is usefull if there are large long-time fluctuations of the signal. see (*Tassieri, M., Evans, R., Warren, R. L., Bailey, N. J. & Cooper, J. M. Microrheology with optical tweezers: data analysis. New Journal of Physics 14, 115032 (2012).*) for details. 

In [ ]:
# set time interval
dt = t[1]
#compute correlations 
taux, Ax, Ay = compute_autocorrelation(xpos,ypos,dt) #for 2^2 time lags
itaux, iAx, iAy = ia_autocorr(xpos,ypos,31, 0.0019968) #for 1.5^n time lags, good if there are  long-time fluctuations in the data
kappax,kappay= get_stiffness(t,1000*xpos,1000*ypos,25) #Getting the trap stiffness therough the equipartition theorem (see the manuscript)
#plot all
plt.semilogx(taux,Ax,'o',color='r',label='x')
plt.semilogx(taux,Ay,'o',color='b',label='y')
plt.semilogx(itaux,iAx,'o',color='g',label='x')
plt.semilogx(itaux,iAy,'o',color='y',label='y')
plt.xlabel('Tau (s)',Fontsize=20)
plt.ylabel('A(tau)',Fontsize=20)
plt.legend()
plt.savefig(fname+'Autocorrelation.png',bbox_inches='tight')
np.savetxt(fname+"Autocorr.txt", np.transpose(np.array([taux,Ax,Ay])))
np.savetxt(fname+"Autocorr_IA.txt", np.transpose(np.array([itaux,iAx,iAy])))

# Fitting the Autocorrelation functions in x and y
We use a three-exponential functions to fit the ACF. The number of exponentials is not fixed, you can increase if the fitting is not adequate. see (*Tassieri, M., Evans, R., Warren, R. L., Bailey, N. J. & Cooper, J. M. Microrheology with optical tweezers: data analysis. New Journal of Physics 14, 115032 (2012).*) for details. 

In [ ]:
# define fitting function
def PMRfit(tau, G1,l1,gamma1,G2,l2,gamma2,G3,l3,gamma3):
    return G1*np.exp(-(l1*tau)**gamma1)+G2*np.exp(-(l2*tau)**gamma2)+G3*np.exp(-(l3*tau)**gamma3) 

#initiate fiting object
modely = lmfit.Model(PMRfit)
params = modely.make_params(G1=1,l1=0.001,gamma1=1.,G2=1,l2=0.01,gamma2=1.,G3=1,l3=0.1,gamma3=1.)
params['G1'].set(value=1.0, vary=True,max=5, min=0.0)
params['l1'].set(value=1., max=10000000.0, min=0.0)
params['gamma1'].set(value=1., vary=True,max=10, min=0.0)
params['G2'].set(value=.0, vary=True,max=5, min=0.0)
params['l2'].set(value=10, vary=True,max=1000000, min=0.0)
params['gamma2'].set(value=1., vary=True,max=10, min=0.0)
params['G3'].set(value=0, vary=True,max=5, min=0.0)
params['l3'].set(value=.01, vary=True,max=1000000, min=0.0)
params['gamma3'].set(value=1, vary=True,max=10, min=0.0)

params['l1'].set(min=1.e-8,max=+np.inf)

#fit
fitresy = modely.fit(Ay, tau=taux, params=params, 
                   method='least_squares')
print(fitresy.fit_report())

#plot stuff
plt.semilogx(taux,Ay,'o')

taunewy=np.logspace(-2.5,0,100000)
Atimenewy=modely.eval(fitresy.params,tau=taunewy)
plt.semilogx(taunewy, Atimenewy)

plt.semilogx(taux, fitresy.init_fit)
plt.xlabel(r'$\tau (s)$')
plt.ylabel(r'$G(\tau)$')
plt.savefig('Autocorrelation_IA_y.png',bbox_inches='tight')

#save data as text files
np.savetxt(fname+'_Autocorrelation_fit_y.txt', np.transpose([taux, Ay, fitresy.best_fit]))

In [ ]:
# initiate fitting object
modelx = lmfit.Model(PMRfit)
params = modely.make_params(G1=1,l1=0.001,gamma1=1.,G2=1,l2=0.01,gamma2=1.,G3=1,l3=0.1,gamma3=1.)
params['G1'].set(value=1.0, vary=True,max=5, min=0.0)
params['l1'].set(value=1., max=10000000.0, min=0.0)
params['gamma1'].set(value=1., vary=True,max=10, min=0.0)
params['G2'].set(value=.0, vary=True,max=5, min=0.0)
params['l2'].set(value=10, vary=True,max=1000000, min=0.0)
params['gamma2'].set(value=1., vary=True,max=10, min=0.0)
params['G3'].set(value=0, vary=True,max=5, min=0.0)
params['l3'].set(value=.01, vary=True,max=1000000, min=0.0)
params['gamma3'].set(value=1, vary=True,max=10, min=0.0)

#fit
fitresx = modelx.fit(Ax, tau=taux, params=params, 
                   method='least_squares')
print(fitresy.fit_report())

#plot all
plt.semilogx(taux,Ax,'o')
taunewx=np.logspace(-2.5,0,100000)
Atimenewx=modely.eval(fitresx.params,tau=taunewx)
plt.semilogx(taunewx, Atimenewx)
plt.semilogx(taux, fitresx.init_fit)
plt.xlabel(r'$\tau (s)$')
plt.ylabel(r'$G(\tau)$')
plt.savefig('Autocorrelation_IA_x.png',bbox_inches='tight')
#save data as text file
np.savetxt(fname+'_Autocorrelation_fit_x.txt', np.transpose([taux, Ax, fitresx.best_fit]))


In [ ]:
plt.semilogx(taux,Ay,'o')
fsmooth = interp1d(taux, Ay, kind='cubic')
trawy=np.logspace(-2.5,0,10000)
Arawy= fsmooth(trawy)
plt.semilogx(trawy,Arawy,'o')

plt.semilogx(taux,Ax,'o')
fsmooth = interp1d(taux, Ax, kind='cubic')
trawx=np.logspace(-2.5,0,10000)
Arawx= fsmooth(trawy)
plt.semilogx(trawy,Arawy,'o')

iAy_tosmooth = np.insert(iAy,0,1)
iAx_tosmooth = np.insert(iAx,0,1)
itaux_tosmooth = np.insert(itaux,0,0)


plt.semilogx(itaux_tosmooth,iAx_tosmooth,'o')
fsmooth = interp1d(itaux_tosmooth, iAx_tosmooth, kind='cubic')
itrawx=np.logspace(-2.5,0,100000)
iArawx= fsmooth(itrawx)
plt.semilogx(itrawx,iArawx,'o')

plt.semilogx(itaux_tosmooth,iAy_tosmooth,'o')
fsmooth = interp1d(itaux_tosmooth, iAy_tosmooth, kind='cubic')
itrawy=np.logspace(-2.5,0,100000)
iArawy= fsmooth(itrawy)
plt.semilogx(itrawy,iArawy,'o')

# Calculating the Rheological moduli
We call the user-defined function PMR to calculate G' and G''. PMR caclulates the rheological properties by first performing a discrete Fourier transform of the Autocorrelation function and then uses the formula $$ G^*(\omega) =\frac{\kappa}{6\pi a}  (\frac{i\omega A(\omega)}{1-i\omega A(\omega)})$$

In [ ]:
# for fitted ACF

omegasy,Afreqy=PMR(taunewy,Atimenewy,1.0,0.0)
omegas1y,Afreq1y= PMR(trawy,Arawy,1.0,0.0)

Pomega=1-1j*omegasy*Afreqy
Gfit =  (kappay/(6*np.pi*0.5e-6))*(1./Pomega-1) 

plt.loglog(omegasy,Gfit.real,'o',label='Storage Modulus G\' fit',alpha=0.5)
plt.loglog(omegasy,Gfit.imag,'o',label='Loss Modulus G\'\' fit',alpha=0.5)
w = np.array([39062,39062])
plt.loglog(w,np.array([0,1e5]))

#w = np.logspace(1,1.6,100)
#plt.loglog(w,(w**1)/10)
#plt.text(10, 2, '1',fontsize=14)
#plt.text(200, 0.2, '2',fontsize=14)
myplot('$\omega$ (rad/s)','G\', G\'\' (Pa)')
plt.savefig('Moduli_Passive_fit_y'+fname+'.png',bbox_inches='tight')
#plt.ylim(0.00000000001,10)

In [ ]:
#for Raw ACF

Pomega1=1-1j*omegas1y*Afreq1y
G =  (kappay/(6*np.pi*0.5e-6))*(1./Pomega1-1) 
plt.loglog(omegas1y,G.real,'o',label='Storage Modulus G\'',alpha=0.5)
plt.loglog(omegas1y,G.imag,'o',label='Loss Modulus G\'\'',alpha=0.5)
plt.loglog(omegasy,Gfit.real,'o',label='Storage Modulus G\' fit',alpha=0.5)
plt.loglog(omegasy,Gfit.imag,'o',label='Loss Modulus G\'\' fit',alpha=0.5)
w = np.array([39062,39062])
plt.loglog(w,np.array([0,1e5]))
#plt.loglog(w,(w**2)/100000)

#w = np.logspace(1,1.6,100)
#plt.loglog(w,(w**1)/10)
#plt.text(10, 2, '1',fontsize=14)
#plt.text(200, 0.2, '2',fontsize=14)
myplot('$\omega$ (rad/s)','G\', G\'\' (Pa)')
plt.savefig('Moduli_Passive_all_y'+fname+'.png',bbox_inches='tight')
plt.xlim(.1,100)

In [ ]:
plt.loglog(omegasy,Gfit.imag/omegasy,'o',label='Viscosity',alpha=0.5)
myplot('$\omega$ (rad/s)','Viscosity (Pa.s)')
plt.savefig('Viscosity_y'+fname+'.png',bbox_inches='tight')
np.savetxt(fname+'moduli-y.txt',np.transpose([omegasy,Gfit.real, Gfit.imag, omegas1y, G.real,G.imag, Gfit.imag/omegasy]),delimiter='	',fmt='%1.9f')


In [ ]:
# For fitted ACF
omegasx,Afreqx=PMR(taunewx,Atimenewx,1.0,0.0)
omegas1x,Afreq1x= PMR(trawx,Arawx,1.0,0.0)

Pomega=1-1j*omegasx*Afreqx
Gfit =  (kappax/(6*np.pi*0.5e-6))*(1./Pomega-1) 

plt.loglog(omegasx,Gfit.real,'o',label='Storage Modulus G\' fit',alpha=0.5)
plt.loglog(omegasx,Gfit.imag,'o',label='Loss Modulus G\'\' fit',alpha=0.5)

#plt.loglog(w,(w**2)/100000)
w = np.array([39062,39062])
plt.loglog(w,np.array([0,1e5]))
#w = np.logspace(1,1.6,100)
#plt.loglog(w,(w**1)/10)
#plt.text(10, 2, '1',fontsize=14)
#plt.text(200, 0.2, '2',fontsize=14)
myplot('$\omega$ (rad/s)','G\', G\'\' (Pa)')
plt.savefig('Moduli_Passive_fit_x'+fname+'.png',bbox_inches='tight')
#plt.ylim(0.00000000001,10)

In [ ]:
# For raw ACF
Pomega1=1-1j*omegas1x*Afreq1x
G =  (kappax/(6*np.pi*0.5e-6))*(1./Pomega1-1) 
plt.loglog(omegas1x,G.real,'o-',label='Storage Modulus G\'',alpha=0.5)
plt.loglog(omegas1x,G.imag,'o-',label='Loss Modulus G\'\'',alpha=0.5)
plt.loglog(omegasx,Gfit.real,'o',label='Storage Modulus G\' fit',alpha=0.5)
plt.loglog(omegasx,Gfit.imag,'o',label='Loss Modulus G\'\' fit',alpha=0.5)
w = np.array([39062,39062])
plt.loglog(w,np.array([0,1e5]))
#plt.loglog(w,(w**2)/100000)

#w = np.logspace(1,1.6,100)
#plt.loglog(w,(w**1)/10)
#plt.text(10, 2, '1',fontsize=14)
#plt.text(200, 0.2, '2',fontsize=14)
myplot('$\omega$ (rad/s)','G\', G\'\' (Pa)')
plt.savefig('Moduli_Passive_all_x'+fname+'.png',bbox_inches='tight')
plt.xlim(.1,100)

In [ ]:
plt.loglog(omegasy,Gfit.imag/omegasx,'o',label='Viscosity',alpha=0.5)
myplot('$\omega$ (rad/s)','Viscosity (Pa.s)')
plt.savefig('Viscosity_x'+fname+'.png',bbox_inches='tight')
np.savetxt(fname+'moduli-x.txt',np.transpose([omegasx,Gfit.real, Gfit.imag, omegas1x, G.real,G.imag, Gfit.imag/omegasx]),delimiter='	',fmt='%1.9f')
